<a href="https://colab.research.google.com/github/pratikshaa12/Generative-AI-Projects/blob/main/LegalDocumentAnalyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install fitz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.4/95.4 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.1/528.1 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 108.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.8 MB/s eta 0:00:00


In [5]:
pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 63.5 MB/s eta 0:00:00


In [6]:
pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 95.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [8]:
pip install transformers

In [9]:
pip install spacy

In [10]:
import fitz
import torch
from transformers import AutoTokenizer, AutoModelForPreTraining,pipeline
import spacy
from google.colab import files

In [14]:
from transformers import pipeline


In [17]:
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

def extract_text_from_pdf(pdf_path):
  doc= fitz.open(pdf_path)
  text= " "
  for page in doc:
    text += page.get_text("text") + "\n"
  return text
pdf_text=  extract_text_from_pdf(file_name)
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")
model = AutoModelForPreTraining.from_pretrained("nlpaueb/legal-bert-base-uncased")
nlp = spacy.load("en_core_web_sm")
summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=-1)

def generate_summary(text):
  summary=summarizer(text, max_length=90, min_length=50, do_sample=False)
  return summary[0]["summary_text"]

def extract_entities(text):
  doc= nlp(text)
  entities={ent.label_: [] for ent in doc.ents}
  for ent in doc.ents:
      entities[ent.label_].append(ent.text)
  return entities

def extract_clauses(text):
    clauses = {
        "Payment Terms":[],
        "Confidentiality":[],
        "Termination": [],
        "Governing Law": []
    }
    for line in text.split("\n"):
        if "pay" in line.lower() or "compensation" in line.lower():
            clauses["Payment Terms"].append(line)
        elif "confidential" in line.lower() or "disclose" in line.lower():
            clauses["Confidentiality"].append(line)
        elif "terminate" in line.lower():
            clauses["Termination"].append(line)
        elif "law" in line.lower() or "jurisdiction" in line.lower():
            clauses["Governing Law"].append(line)
    return clauses

def analyze_risks(clauses):
    risk_keywords = ["breach", "liability", "penalty", "damages", "termination without cause"]
    risks= []
    for category, clause_list in clauses.items():
        for clause in clause_list:
            if any(word in clause.lower() for word in risk_keywords):
                risks.append(f"Possible Risk in {category}: {clause}")
        return risks
summary= generate_summary(pdf_text)
entities= extract_entities(pdf_text)
clauses= extract_clauses (pdf_text)
risks= analyze_risks(clauses)

print("\n Summary of The legal Document")
print(summary)

print("\n Named Entities (Parties, Dates, Amounts, ETC)")
for key, values in entities.items():
    if values:
      print(f"{key}: {', '.join(values)}")
print("\n Extracted Clauses")
for key, values in clauses.items():
    if values:
      print(f"\n {key}")
      for clause in values:
          print(f"\n {key}:")
          for clause in values:
              print(f"-{clause}")
print("\n Risk Analysis")
if risks:
    for risk in risks:
        print(risk)
else:
    print("No high risk clauses detected")



Saving legal_contract_document.pdf to legal_contract_document (3).pdf


Device set to use cpu



 Summary of The legal Document
The failure of either party to enforce any provision shall not be construed as a waiver of such provision. Neither party shall be liable for any indirect or consequential damages arising from this Agreement. This Agreement shall be governed by and construed in accordance with the laws of the jurisdiction in which it is entered.

 Named Entities (Parties, Dates, Amounts, ETC)
PRODUCT: Agreement, Agreement
DATE: 30) days

 Extracted Clauses

 Termination

 Termination:
-Either party may terminate this Agreement upon thirty (30) days prior written notice. The failure of

 Governing Law

 Governing Law:
-Agreement shall be governed by and construed in accordance with the laws of the jurisdiction.

 Risk Analysis
No high risk clauses detected
